In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForSequenceClassification.from_pretrained(
    "ChaiML/gpt2_base_retry_and_continue_12m_reward_model"
)
tokenizer.pad_token_id = 50256
tokenizer.truncation_side = "left"
tokenizer.padding_side = "right"

In [21]:
tokens = tokenizer(
    ["Hello how are you?", "You are very bad."],
    # "Hello how are you?",
    return_tensors="pt",
    return_attention_mask=True,
    padding="longest",
    truncation=True,
    max_length=256,
)

with torch.no_grad():
    reward = model(**tokens)

In [22]:
reward.logits.argmax(-1)

tensor([1, 1])

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(
    reward_name
), AutoTokenizer.from_pretrained(reward_name)

question = "I just came out of from jail, any suggestion of my future?"
helpful = "It's great to hear that you have been released from jail."
bad = "Go back to jail you criminal."

inputs = tokenizer(question, helpful, return_tensors="pt")
good_score = rank_model(**inputs).logits[0].cpu().detach()

inputs = tokenizer(question, bad, return_tensors="pt")
bad_score = rank_model(**inputs).logits[0].cpu().detach()
print(good_score > bad_score)  # tensor([True])

tensor([True])


In [4]:
good_score, bad_score

(tensor([-2.5912]), tensor([-4.1492]))